# Data Migration
**Prerequisites**
- Access to a CDF Project.
- Know how to use a terminal, so you can run `pygen` from the command line to 
  generate the SDK.
- Knowledge of your the data and data models.

In [1]:
import warnings
warnings.filterwarnings('ignore')
# This is just to enable improting the generated SDK from the examples folder in the pygen repository
import sys
from tests.constants import REPO_ROOT
sys.path.append(str(REPO_ROOT / "examples" ))

## Introduction to Problem

In the development of a solution, it is common that you move from one data model to another. Typically, the physicaly storage of data, the containers, stays the same, or are extended, 
and only the views of a data model change. In this case, there is no need for a data migration and all you do is update the solution code for the new model. 

If you, however, need to change how the data is stored in the containers, you need to move the data from one model to the other. A common to do this is to use CDF Transformations. An alternative
ways is to use `pygen` for data migration.

#### Advantage of CDF Transformations
* Can handle large volume of data, millions of instances.
* You can write the transformations in the UI using SQL.


#### Advantages of using `pygen` 
* Edges are automatically created.
* Can handle medium volume of data, hundreds of thousands of instances.
* Support doing very custom migration (what ever you can do in Python code).

**Why** would you **chose `pygen`** over **CDF Transformations**? If you are more comfortable with `Python` than writing `SQL`, you have data models with a of one-to-many edges, the number of nodes and edges (instances) is in the order of hundreds of thousands.




## Use Case

The use case we use in this guide is moving from one Asset Performance Model to another. 

We wil refer to the model we are moving away from as the **source** model and
the model we are moving to as the **destination** model.

**Source** Model


<img src="images/source_model.png" width="400">

**Destination Model**:


<img src="images/destination_model.png" width="400">

As we see in the illustratoins above (click to enlarge), we have many more edges in the destination than the source model, 
whiche means this is a good use case for using `pygen` for the data migration.

In this guide, we will not do the entire migration, but instead focus on the `APM_Template` to `Template` migration.

## Generating SDKs

For demo purposes we will generate the SDKs in this notebook, however, depending on your use case it might be useful to use the `pygen` CLI to generate the SDKs.

Migration scrips are often a one of, meaning you are not expected to regenerat the SDK as the model changes, thus if you generate it locally, you can do changes
to the generated code as you are not expecting this to change later. This can be useful if you want to do some customization to the generated code.

Note that we have set up a `config.toml` with credentials to connect to CDF.

In [1]:
from cognite.pygen import load_cognite_client_from_toml, generate_sdk_notebook

In [2]:
client = load_cognite_client_from_toml("config.toml")

In [3]:
source = generate_sdk_notebook(("APM_AppData_4", "APM_AppData_4", "7"), client)

Successfully retrieved data model(s) ('APM_AppData_4', 'APM_AppData_4', '7')
Writing SDK to C:\Users\ANDERS~1\AppData\Local\Temp\pygen
Done!
Added C:\Users\ANDERS~1\AppData\Local\Temp\pygen to sys.path to enable import
Imported apm_app_data_4.client


In [4]:
destination = generate_sdk_notebook(("IntegrationTestsImmutable", "ApmAppData", "v3"), client)

Successfully retrieved data model(s) ('IntegrationTestsImmutable', 'ApmAppData', 'v3')
Writing SDK to C:\Users\ANDERS~1\AppData\Local\Temp\pygen
Done!
C:\Users\ANDERS~1\AppData\Local\Temp\pygen already in sys.path
Imported apm_app_data.client


## Retrieving Data

It is common to store the nodes and edges, i.e., instances, in a differen space than the data model. To only get instances from an instance space, `sourceSpace`, 
we filter for it. In addition, it important that we retrieve all edges so we can connect the `templates` with the `templates_items`.

In [8]:
templates = source.apm_template.list(space="sourceSpace", retrieve_edges=True, limit=-1)
template_items = source.apm_template_item.list(space="sourceSpace", limit=-1)

In [9]:
len(templates), len(template_items)

(50, 105)

## Convesting the Data

We do the conversion in in the following steps

1. We show some of the data of the source templates. This is usefult to have as a reference when writing the transformation code
2. We put the `APMTemplateItem`s into a dictionary, `template_items_by_external_id` so we can easily reference it.
3. We include a few other dictionaries, `visiblity_by_item_external_id`, `schedules_by_item_external_id`, `measurements_by_external_id`, and `visiblity_by_external_id`.
   This is to show how we can include data from different sources. 
5. We iterate through all the `APMTemplates` and do the transformation for each `template_item`(s)s and `template`


In [37]:
# Importing the new destination data classes
from apm_app_data.client.data_classes import TemplateApply, TemplateItemApply, TemplateApplyList

In [12]:
templates.to_pandas().head(5)

,external_id,assigned_to,created_by_external_id,is_archived,root_location_external_id,status,title,updated_by_external_id,template_items,space,version,last_updated_time,created_time,deleted_time
0,108c5106-f638-4d9c-8ef2-4b470ed94495,None,80d32fb4-bcbc-44ae-abed-fee53b148d82,False,31d9f834-a1ab-452a-b2a5-875f0e9f384a,OnHold,good,f1e7d13d-3fd2-42e4-bebc-68ad2aa905eb,"[bef8179e-9e0c-44a8-8577-cf6e5ce0bd8f, 479ef47...",sourceSpace,1,2023-11-05 10:53:03.351000+00:00,2023-11-05 10:53:03.351000+00:00,None
1,32f92434-e5fb-49c1-b4c4-1a454ed699ea,None,6dbb225f-5059-4ff2-9543-7ab400e235e9,False,dac4077b-61d6-445c-a4f4-f6f3247125ab,OnHold,three,df561fd7-dad6-484e-95f7-ae967fb00ed3,"[f6ce45c7-b5a6-4a3b-b549-91e4a05580a7, bbbdda6...",sourceSpace,1,2023-11-05 10:53:03.351000+00:00,2023-11-05 10:53:03.351000+00:00,None
2,6b072855-8408-4c21-b1a3-5caf4bd22204,None,863a0cd1-f55e-4529-ac50-34090596a279,True,1b582c43-bc07-4ab7-959e-cf7e434711a0,Completed,also,37d17cd1-2bbe-416a-b7b1-111e44c64b36,"[8b3670e5-a6e4-4636-8066-ee54c0a52582, 29989d8...",sourceSpace,1,2023-11-05 10:53:03.351000+00:00,2023-11-05 10:53:03.351000+00:00,None
3,95c51139-438b-4fd8-a5ce-b80dbcd77db4,None,ac5c7d3f-564b-44d2-bdc6-6eaf23b075ce,True,1b582c43-bc07-4ab7-959e-cf7e434711a0,Completed,likely,caba29e9-fa98-438d-b5b7-6d43677764d3,"[cc2adb6c-34e8-4258-8634-3213c789854c, ffede5f...",sourceSpace,1,2023-11-05 10:53:03.351000+00:00,2023-11-05 10:53:03.351000+00:00,None
4,9544b2c9-9abe-4a9b-b106-3db9510363c2,None,b3fd00cb-8b06-4365-aac5-b0fcf6f8044c,True,41ad2ff1-88b6-4c45-9bfd-a76d0c0ba9c6,Cancelled,thing,ac5c7d3f-564b-44d2-bdc6-6eaf23b075ce,"[502250e9-237b-4c87-85a3-ee100601bdba, 1881a55...",sourceSpace,1,2023-11-05 10:53:03.351000+00:00,2023-11-05 10:53:03.351000+00:00,None


In [13]:
template_items.to_pandas().sample(5)

,external_id,asset_external_id,created_by_external_id,labels,order,template_external_id,title,updated_by_external_id,space,version,last_updated_time,created_time,deleted_time
91,3396be3e-d4dd-4546-b6a2-e73c1036431f,3dccb9c8-92e4-43e6-9b09-cbb7dd767cbf,146b3ec8-ec8e-48c3-a68d-dbdcd53ef962,[#f495e3],4142,9199d796-9bf6-4fc8-9376-5101d533009a,suffer,None,sourceSpace,1,2023-11-05 10:53:03.351000+00:00,2023-11-05 10:53:03.351000+00:00,None
49,44731c25-e7e6-4b2c-aae5-0d2b495f17f9,95d2dae0-a976-4400-8cbf-e6870790f00f,05b39884-406c-4523-a302-24834ec842c4,"[#f4d89f, #dd6388, #f495e3]",4142,4a9c75d8-3213-4e6f-9c2c-c8750ecdb4a9,newspaper,None,sourceSpace,1,2023-11-05 10:53:03.351000+00:00,2023-11-05 10:53:03.351000+00:00,None
73,3130d24c-b528-46de-abe3-4e1e86c4dc90,3233aa0e-9c01-433e-9706-71352468aa80,0f53f902-a251-4b66-a9ae-7eb4eb6890da,[#f495e3],8088,6fe2a770-70d6-4ca3-85db-76ee5369b1f2,feeling,None,sourceSpace,1,2023-11-05 10:53:03.351000+00:00,2023-11-05 10:53:03.351000+00:00,None
4,f6ce45c7-b5a6-4a3b-b549-91e4a05580a7,f4f6daa5-1241-48a8-99a6-5c594c4bbe1f,66f3e1db-db6d-4283-963c-d6284217d06c,"[#7fe0a9, #f495e3, #f4d89f]",765,32f92434-e5fb-49c1-b4c4-1a454ed699ea,person,None,sourceSpace,1,2023-11-05 10:53:03.351000+00:00,2023-11-05 10:53:03.351000+00:00,None
90,35b5ff44-b05e-4979-9af2-94932a750797,c9da1796-ebc7-4205-99dc-dcdcc1c5f252,891ffc81-5d83-4cfa-b306-5ede51a5293f,"[#f495e3, #dd6388]",8193,9199d796-9bf6-4fc8-9376-5101d533009a,main,None,sourceSpace,1,2023-11-05 10:53:03.351000+00:00,2023-11-05 10:53:03.351000+00:00,None


In [20]:
template_item_by_external_id = {item.external_id: item for item in template_items}

# The visiblity, schedules, and measurements are changes to the new template,
# and thus the source of these needs to come from elsewhere. The point of including 
# these data structers is to show how 
visiblity_by_item_external_id: dict[str, str] = {}
schedules_by_item_external_id: dict[str, list[str]] = {}
measurements_by_external_id: dict[str, list[str]] = {}

visiblity_by_external_id: dict[str, str] = {}

In [24]:
destination_templates = TemplateApplyList([])
for template in templates:
    destination_items = []
    for item_external_id in template.template_items or []:
        item = template_item_by_external_id[item_external_id]
        destination_item = TemplateItemApply(
            external_id=item.external_id,
            space="destinationSpace",
            title=item.title,
            labels=item.labels,
            visibility=visiblity_by_item_external_id.get(item.external_id),
            created_by=item.created_by_external_id,
            updated_by=item.updated_by_external_id,
            is_archived=template.is_archived,
            order=item.order,
            asset=item.asset_external_id,
            schedules=schedules_by_item_external_id.get(item.external_id),
            measurements=measurements_by_external_id.get(item.external_id),            
        )
        destination_items.append(destination_item)
    
    destination_template = TemplateApply(
        external_id=template.external_id,
        space="destinationSpace",
        title=template.title,
        visibility=visiblity_by_external_id.get(template.external_id),
        created_by=template.created_by_external_id,
        updated_by=template.updated_by_external_id,
        is_archived=template.is_archived,
        status=template.status,
        labels=list({label for item in destination_items for label in item.labels}),
        root_location=template.root_location_external_id,
        assigned_to=template.assigned_to,
        template_items=destination_items,
    )
    destination_templates.append(destination_template)

## How <code>pygen</code> Creates Edges

`pygen` create edges by looking at the relationships in the generated data classes. 

For example, in the current example we have the `TemplateItem` node which have the fields `schedules` and `measurements`. When writing to CDF `pygen` iterates through all the `TemplateItems` and check these fields, and if there is a `string` it is assumed to be the `external_id` of the end node, and `pygen` creates an edge to it. If it is an `object` that object is assumed to be the end node and `pygen` creates an edge to it.

We can inspect what pygen is doing by calling the `.to_instances_apply()` method and check the results.



In [26]:
destination_templates[0]

,value
space,destinationSpace
external_id,108c5106-f638-4d9c-8ef2-4b470ed94495
existing_version,None
title,good
description,None
labels,"[#f495e3, #f4d89f, #7fe0a9, #8516fc, #dd6388]"
visibility,None
created_by,80d32fb4-bcbc-44ae-abed-fee53b148d82
updated_by,f1e7d13d-3fd2-42e4-bebc-68ad2aa905eb
is_archived,False


In [31]:
len(destination_templates[0].template_items)

4

In [27]:
instances = destination_templates[0].to_instances_apply()

In [28]:
instances.nodes

,instance_type,space,external_id,sources
0,node,destinationSpace,108c5106-f638-4d9c-8ef2-4b470ed94495,"[{'properties': {'title': 'good', 'labels': ['..."
1,node,destinationSpace,bef8179e-9e0c-44a8-8577-cf6e5ce0bd8f,"[{'properties': {'title': 'half', 'labels': ['..."
2,node,destinationSpace,479ef472-a2c1-4f03-b38d-7e29e4f90227,"[{'properties': {'title': 'agreement', 'labels..."
3,node,destinationSpace,a92cfb02-09cd-4b47-b870-a7aded8aed07,"[{'properties': {'title': 'behind', 'labels': ..."
4,node,destinationSpace,54e8a840-624d-41cb-91a3-1b978e292565,"[{'properties': {'title': 'wall', 'labels': ['..."


In [29]:
instances.edges

,instance_type,space,external_id,type,start_node,end_node
0,edge,destinationSpace,108c5106-f638-4d9c-8ef2-4b470ed94495:bef8179e-...,"{'space': 'cdf_apm', 'external_id': 'reference...","{'space': 'destinationSpace', 'external_id': '...","{'space': 'destinationSpace', 'external_id': '..."
1,edge,destinationSpace,108c5106-f638-4d9c-8ef2-4b470ed94495:479ef472-...,"{'space': 'cdf_apm', 'external_id': 'reference...","{'space': 'destinationSpace', 'external_id': '...","{'space': 'destinationSpace', 'external_id': '..."
2,edge,destinationSpace,108c5106-f638-4d9c-8ef2-4b470ed94495:a92cfb02-...,"{'space': 'cdf_apm', 'external_id': 'reference...","{'space': 'destinationSpace', 'external_id': '...","{'space': 'destinationSpace', 'external_id': '..."
3,edge,destinationSpace,108c5106-f638-4d9c-8ef2-4b470ed94495:54e8a840-...,"{'space': 'cdf_apm', 'external_id': 'reference...","{'space': 'destinationSpace', 'external_id': '...","{'space': 'destinationSpace', 'external_id': '..."


We can see here that for one `Template` item with 4 `TemplateItem`s `pygen` creates one node for each `=5`, and an edge connecting each of the `TemplateItem` to `Template` `=4`.

In [39]:
all_instances = destination_templates.to_instances_apply()

By doing the `.to_instances_apply` call on the all the destination `Templates` we can see how 
many nodes and edges we are expecting to create.

In [41]:
len(all_instances.nodes), len(all_instances.edges)

(155, 105)

Note that for the use case we are using in this example one of the changes to the new model is that a lot of the relationships are made explicit. For example, in the old template we have `root_location_external_id`, while now we have direct link `root_location`. Due to the way `pygen` automatically creates edges this becomes an easier migration as if `pygen` knows that there should be an edge and have a `string` it is assumed to be an `external_id`. 


## Writing Data

In [42]:
# In case the space doesn't exists. This cell is hidden from the tutorial
from cognite.client import data_modeling as dm
client.data_modeling.spaces.apply(dm.SpaceApply("destinationSpace", name="Destination space", description="For the data migration tutorial"));

We are no ready to write the new `Template` data to the new model.

We use the `destination` client and call `.apply` on the `template`.

In [44]:
results = destination.template.apply(destination_templates)

CogniteAPIError: Cannot auto-create a direct relation target in node '108c5106-f638-4d9c-8ef2-4b470ed94495' because it has a container constraint. The target node must already exist and have data in the specified container. | code: 400 | X-Request-ID: c714095b-5925-9737-b118-c38d2942eeee
The API Failed to process some items.
Successful (2xx): []
Unknown (5xx): []
Failed (4xx): ['108c5106-f638-4d9c-8ef2-4b470ed94495', 'bef8179e-9e0c-44a8-8577-cf6e5ce0bd8f', '479ef472-a2c1-4f03-b38d-7e29e4f90227', 'a92cfb02-09cd-4b47-b870-a7aded8aed07', '54e8a840-624d-41cb-91a3-1b978e292565', '32f92434-e5fb-49c1-b4c4-1a454ed699ea', 'f6ce45c7-b5a6-4a3b-b549-91e4a05580a7', 'bbbdda6e-b6d7-41e8-a722-3ed00031a9ba', '7a11b48e-a547-43cc-b907-83ef6ef363d3', '6b072855-8408-4c21-b1a3-5caf4bd22204', '8b3670e5-a6e4-4636-8066-ee54c0a52582', '29989d84-cd33-484f-a46f-b169249bdb14', 'e85f295b-da0f-48c5-8ef8-c25a1921047c', '95c51139-438b-4fd8-a5ce-b80dbcd77db4', 'cc2adb6c-34e8-4258-8634-3213c789854c', 'ffede5fa-f5ce-4f4a

Note that it is not necessary to do a call to `destination.template_items.apply(...)` as the above call iterates recursively through all linked objects and finds the nodes and edges.

We can verify that the data has been uploaded correctly by calling the `.list()` method on the destination client.

In [45]:
destination.template.list()

,external_id,title,description,labels,visibility,created_by,updated_by,is_archived,status,root_location,assigned_to,template_items,space,version,last_updated_time,created_time,deleted_time
